In [ ]:
#cópia
import random as rd
import time

try:
    import sim
except:
    print ('--------------------------------------------------------------')
    print ('"sim.py" could not be imported. This means very probably that')
    print ('either "sim.py" or the remoteApi library could not be found.')
    print ('Make sure both are in the same folder as this file,')
    print ('or appropriately adjust the file "sim.py"')
    print ('--------------------------------------------------------------')
    print ('')


print ('Program Started')
sim.simxFinish(-1)  # just in case, close all opened connections
clientID=sim.simxStart('127.0.0.1',19999,True,True,5000,5) # Connect to CoppeliaSim

if clientID!=-1:
    print ('Connected to remote API server')
    GoalGlobal = Model('GoalGlobal', clientID)
    
    robotname = 'kobuki'
    robo = Model(robotname, clientID)
    returnCode, robotHandle = sim.simxGetObjectHandle(clientID, robotname, sim.simx_opmode_one_shot_wait)
    returnCode, leftMotor = sim.simxGetObjectHandle(clientID, 'kobuki_leftMotor', sim.simx_opmode_one_shot_wait)
    returnCode, rightMotor = sim.simxGetObjectHandle(clientID, 'kobuki_rightMotor', sim.simx_opmode_one_shot_wait)
    returnCode, robotPos = sim.simxGetObjectPosition(clientID, robotHandle, -1, sim.simx_opmode_streaming)
    returnCode, robotOri = sim.simxGetObjectOrientation(clientID, robotHandle, -1, sim.simx_opmode_streaming)
    
    # handle para os dados do laser
    laser = Model_abs('fastHokuyo', clientID)
    laser_range_data = "hokuyo_range_data"
    laser_angle_data = "hokuyo_angle_data"
    
    # handle para goal global
    returnCode, goalHandle = sim.simxGetObjectHandle(clientID, 'GoalGlobal', sim.simx_opmode_one_shot_wait)
    
    # Transformação do laser para o robo
    Tlr = robo.trans_inv @ laser.trans_global
    
    # Em loop até garantir que as leituras serão válidas
    returnCode = 1
    while returnCode != 0:
        returnCode, range_data = sim.simxGetStringSignal(clientID, laser_range_data, sim.simx_opmode_buffer)
       
    # Prosseguindo com as leituras
    raw_range_data, raw_angle_data = readSensorData(laser_range_data, laser_angle_data)
    laser_data = np.array([raw_angle_data, raw_range_data]).T
    
    # Goal
    pgoal = np.array([GoalGlobal.posicao[0], GoalGlobal.posicao[1]])
    
    # Variáveis e Auxiliares
    robo_posicoes = []
    x_laser = []
    y_laser = []
    distanceGoal = np.linalg.norm(np.array([GoalGlobal.posicao])- np.array([robo.posicao]))
    linhas = []
    
    # Específico do robô
    L = 0.230
    r = 0.035
    maxv = 0.2
    maxw = np.deg2rad(45)
    
    # Caminhos
    caminho1 = np.array([[0.4, -4.45], [0.75, -1], [1.3, 3.8], [0.8, -1], [-1.55, -0.4], [-4.1, -1], [-4.475, 1.025]])
    caminho2 = np.array([[0.4, -4.45], [0.75, -1], [1.3, 3.8], [0.8, -1]])
    caminho3 = np.array([[0.4, -4.45], [0.75, -1], [1.3, 3.8]])
    caminho4 = np.array([[0.4, -4.45]])
    
    for caminho in caminho2:
        pgoal = caminho
        returnCode = sim.simxSetObjectPosition(clientID, goalHandle, -1, [pgoal[0], pgoal[1], 0], sim.simx_opmode_oneshot_wait)
        distanceGoal = np.linalg.norm(pgoal - robo.posicao[:2])
        
        while (distanceGoal > 0.5):
            # Leitura do laser
            laser.atualiza()
            raw_range_data, raw_angle_data = readSensorData(laser_range_data, laser_angle_data)
            laser_data = np.array([raw_angle_data, raw_range_data]).T
            dados = np.array(get_all_laser_data_ruido(laser_data, laser.trans_global))
            x_laser = np.append(x_laser, dados[:,0])
            y_laser = np.append(y_laser, dados[:,1])
            
            # Lê todos os dados captados pelo laser e atualiza o mapa
            # do occupancy grid
            robo.atualiza()
            for i in range(len(dados)):
                fim_laser = Rz(np.pi/2) @ [(dados[i,0] + map_size[0]/2), (dados[i,1] + map_size[1]/2), 0]
                robo_mapa = Rz(np.pi/2) @ [(robo.posicao[0] + map_size[0]/2), (robo.posicao[1] + map_size[1]/2), 0]
                
                max_cel = len(occupancy_grid) - 1
                robox = min(max_cel, (robo_mapa[0]/cell_size).astype(int)) # posicao x do robo na celula do mapa occ grid
                roboy = min(max_cel, (robo_mapa[1]/cell_size).astype(int)) # posicao y do robo na celula do mapa occ grid
                celulax = min(max_cel, (fim_laser[0]/cell_size).astype(int)) # posicao x do laser na celula do mapa occ grid
                celulay = min(max_cel, (fim_laser[1]/cell_size).astype(int))
                
                occupancy_grid_mapping(np.array([robox, roboy]), np.array([celulax, celulay]))
                
            # Atualizando robo
            robo.atualiza()
            robo_posicoes.append(robo.posicao[:2])
            
            # Roda campos potenciais com controlador de luca e oriolo
            # calculando forças
            atracao = att_force(robo.posicao[:2], pgoal)
            repulsao = rep_force(robo.posicao[:2], [x_laser, y_laser])
            dx, dy = atracao + repulsao
            
            kr = 0.05
            kt = 0.1
            
            v = kr * (dx*np.cos(robo.rotacao[2]) + dy*np.sin(robo.rotacao[2]))
            w = kt * (np.arctan2(dy, dx) - robo.rotacao[2])
            
            v = max(min(v, maxv), -maxv)
            w = max(min(w, maxw), -maxw)
            
            vr = (2*v + L*w) / (2*r)
            vl = (2*v - L*w) / (2*r)
            
            # Envia velocidades calculadas
            sim.simxSetJointTargetVelocity(clientID, leftMotor, vl, sim.simx_opmode_streaming)
            sim.simxSetJointTargetVelocity(clientID, rightMotor, vr, sim.simx_opmode_streaming)
            distanceGoal = np.linalg.norm(pgoal - robo.posicao[:2])
            
    sim.simxSetJointTargetVelocity(clientID, leftMotor, 0, sim.simx_opmode_oneshot_wait)
    sim.simxSetJointTargetVelocity(clientID, rightMotor, 0, sim.simx_opmode_oneshot_wait)
    print ('Program ended')
            

In [ ]:
import math

def sensorToWorld(laser_data, robot_pos, robot_ori):
    xy_w = []
    for measure in laser_data:
        x = measure[1] * math.cos(measure[0])
        y = measure[1] * math.sin(measure[0])
        rot = Rz(robot_ori[2]) @ np.array([[x], [y], [1]])
        xy_w.append([rot[0][0] + robot_pos[0], rot[1][0] + robot_pos[1]])
        
    return xy_w 

def worldToMap(coord, mapCoord, worldCoord):
    px = (mapCoord[1] - mapCoord[0] * (coord[0]-worldCoord[0])) / (worldCoord[1] - worldCoord[0])
    py = (mapCoord[3] - mapCoord[2] * (coord[1]-worldCoord[2])) / (worldCoord[3] - worldCoord[2])
    
    return [math.ceil(px), math.ceil(py)]

def mapToWorld(pix, mapCoord, worldCoord):
    cx = (worldCoord[1] - worldCoord[0]) * (pix[0]*mapCoord[0]) / (mapCoord[1] - mapCoord[0]) + worldCoord[0]
    cy = ((worldCoord[3] - worldCoord[2]) * (pix[1]*mapCoord[2]) / (mapCoord[3] - mapCoord[2])) + worldCoord[2]
    
    return [cx, cy]

def inverseLaserModel(grid, obs_grid):
    if (grid[0] == obs_grid[1] and grid[1] == obs_grid[0]):
            return math.log(0.9/(1-0.9))
    else:
        return math.log(0.1/(1-0.1))
    
def occupancyGrid(obstacles, pose, occGrid):
    pose_grid = worldToMap(pose, mapCoord, worldCoord)
    l0 = math.log(pdesc/(1-pdesc))
    
    for obstacle in obstacles:
        obs_grid = worldToMap(obstacle, mapCoord, worldCoord)
        rr , cc = line(pose_grid[0], pose_grid[1], obs_grid[0], obs_grid[1])
        for cell in list(zip(rr, cc)):
            occGrid[cell] = inverseLaserModel(cell, obs_grid) + occGrid[cell]
    return occGrid 
        

In [ ]:
import math

def sensorToWorld(laser_data, robot_pos, robot_ori):
    xy_w = []
    for measure in laser_data:
        x = measure[1] * math.cos(measure[0])
        y = measure[1] * math.sin(measure[0])
        rot = Rz(robot_ori[2]) @ np.array([[x], [y], [1]])
        xy_w.append([rot[0][0] + robot_pos[0], rot[1][0] + robot_pos[1]])
        
    return xy_w 

def worldToMap(coord, mapCoord, worldCoord):
    px = (mapCoord[1] - mapCoord[0] * (coord[0]-worldCoord[0])) / (worldCoord[1] - worldCoord[0])
    py = (mapCoord[3] - mapCoord[2] * (coord[1]-worldCoord[2])) / (worldCoord[3] - worldCoord[2])
    
    return [math.ceil(px), math.ceil(py)]

def mapToWorld(pix, mapCoord, worldCoord):
    cx = (worldCoord[1] - worldCoord[0]) * (pix[0]*mapCoord[0]) / (mapCoord[1] - mapCoord[0]) + worldCoord[0]
    cy = ((worldCoord[3] - worldCoord[2]) * (pix[1]*mapCoord[2]) / (mapCoord[3] - mapCoord[2])) + worldCoord[2]
    
    return [cx, cy]

def occupancy_grid_mapping2(cel_robo, cel_final, occupancy_grid):
    distanceMax = np.linalg.norm(cel_robo - cel_final)
    rr, cc = line(cel_robo[0], cel_robo[1], cel_final[0], cel_final[1])
    for i in range(len(rr)):
        cel_prob = occupancy_grid[rr[i], cc[i]]
        l_ant = log_odds(cel_prob)
        cel = np.array([rr[i], cc[i]])
        l_cell = l_ant + inverse_range_sensor_model(cel, cel_robo, distanceMax, cel_prob)
        occupancy_grid[rr[i], cc[i]] = inv_log_odds(l_cell)
    return occupancy_grid

def inverseLaserModel(grid, obs_grid):
    if (grid[0] == obs_grid[1] and grid[1] == obs_grid[0]):
            return math.log(0.9/(1-0.9))
    else:
        return math.log(0.1/(1-0.1))
    
def occupancyGrid(obstacles, cel_robo, cel_final, occGrid):

    for obstacle in obstacles:
        obs_grid = worldToMap(obstacle)
        rr , cc = line(cel_robo[0], cel_robo[1], cel_final[0], cel_final[1])
        for cell in list(zip(rr, cc)):
            occGrid[cell] = inverseLaserModel(cell, obs_grid) + occGrid[cell]
    return occGrid 
        
# Cálculo do inverse sensor model
def inverse_range_sensor_model(cel, cel_robot, max_distance, prob):
    max = 5/cell_size
    alpha = 0.2/cell_size
    distanceObs = np.linalg.norm(cel - cel_robot[:2])
    pocc = 0.85
    pfree = 0.1
    locc = log_odds(pocc)
    lfree = log_odds(pfree)
    if distanceObs > min(max, max_distance + alpha/2):
        return log_odds(prob)
    if max_distance < max and abs(distanceObs - max_distance) <= alpha/2:
        return locc
    if distanceObs <= max_distance:
        return lfree

def occupancy_grid_mapping(cel_robo, cel_final, occupancy_grid):
    distanceMax = np.linalg.norm(cel_robo - cel_final)
    rr, cc = line(cel_robo[0], cel_robo[1], cel_final[0], cel_final[1])
    for i in range(len(rr)):
        cel_prob = occupancy_grid[rr[i], cc[i]]
        l_ant = log_odds(cel_prob)
        cel = np.array([rr[i], cc[i]])
        l_cell = l_ant + inverse_range_sensor_model(cel, cel_robo, distanceMax, cel_prob)
        occupancy_grid[rr[i], cc[i]] = inv_log_odds(l_cell)
    return occupancy_grid

# Criando o mapa de ocupação
map_size = np.array([10, 10])
cell_size = 0.1

rows, cols = (map_size/cell_size).astype(int) # Número de linhas e colunas do mapa

# Inicialização do occupancy grid
occupancy_grid = np.full((rows, cols), 0.5)  # Valor inicial 0.5 para representar células desconhecidas

# Conectando-se ao CoppeliaSim
clientID = sim.startSimulation()

if clientID != -1:
    print('Program Started')
    
    robot_controller = RobotController('kobuki')

    robot_controller.start_simulation()
    robot_pos = robot_controller.get_robot_position()
    robot_ori = robot_controller.get_robot_orientation()

    # Lista de objetivos que cobre todo o mapa
    goal_positions = [ [0.4, -4.45, 1], [0.8, -1.8, 1], [1.3, 0, 1], [0.7, 3, 1], [2.5, 4.3, 1]] #  
    
    for goal_position in goal_positions:
        robot_controller.set_goal_position(goal_position)
        
        distance_goal = np.linalg.norm(np.array(goal_position[:2]) - np.array(robot_pos[:2]))
        
        while distance_goal > 0.5:
            # Leitura do laser
            if laser_range_data is not None and laser_angle_data is not None:
                raw_range_data, raw_angle_data = readSensorData() 
                
            laser_data = np.array([raw_angle_data, raw_range_data]).T
        
            laser_transform = Rz(np.pi / 2)
            noisy_data = get_all_laser_data_ruido(laser_data, laser_transform)
            
            laser_data_total = []

            # Atualizando robo
            robot_pos = robot_controller.get_robot_position()
            robot_ori = robot_controller.get_robot_orientation()
            
            occupancy_grid = np.empty((rows, cols))
            
            
            for i in range(len(laser_data)):
                fim_laser = Rz(np.pi/2) @ [(np.array(laser_data[i][0]) + map_size[0]/2), (np.array(laser_data[i][1]) + map_size[1]/2), 0]
                robo_mapa = Rz(np.pi/2) @ [(robot_pos[0] + map_size[0]/2), (robot_pos[1] + map_size[1]/2), 0]
                max_cel = len(occupancy_grid) - 1
                robot_x = min(max_cel, (robo_mapa[0]/cell_size).astype(int)) # posicao x do robo na celula do mapa occ grid
                robot_y = min(max_cel, (robo_mapa[1]/cell_size).astype(int)) # posicao y do robo na celula do mapa occ grid
                celula_x = min(max_cel, (fim_laser[0]/cell_size).astype(int)) # posicao x do laser na celula do mapa occ grid
                celula_y = min(max_cel, (fim_laser[1]/cell_size).astype(int))
                

                #occupancy_grid = occupancy_grid_mapping3(np.array([robot_x, robot_y]), np.array([celula_x, celula_y]), occupancy_grid, cell_size)
                occupancy_grid = occupancy_grid_mapping2(np.array([robot_x, robot_y]), np.array([celula_x, celula_y]), occupancy_grid)
            
            Hlw = transformacao(robot_pos, robot_ori) # Matriz de transformação do laser para o mundo
            
            obs = [] # Lista de obstáculos
            obs_pts = [] # Lista de pontos dos obstáculos
            
            # Calculando forças
            Ft, Fatt, Frep = tt_force(np.array(robot_pos[:2]), np.array(goal_position[:2]), laser_data, obs, obs_pts, Hlw)
                
            dx, dy = Ft

            kr = 0.05
            kt = 0.1
            
            L = 0.381 # Distância entre as rodas
            R = 0.0975 # Raio da roda

            v = kr * (dx * np.cos(robot_ori[2]) + dy * np.sin(robot_ori[2]))
            w = kt * (np.arctan2(dy, dx) - robot_ori[2])

            v = max(min(v, 0.2), -0.2)
            w = max(min(w, np.deg2rad(45)), -np.deg2rad(45))

            # cinemática inversa   
            #vr = (2 * v + 0.23 * w) / (2 * 0.035)
            #vl = (2 * v - 0.23 * w) / (2 * 0.035)
            
            vr = ((2 * v) + (w * L) / 2 * R)
            vl = ((2 * v) + (w * L) / 2 * R)

            # Envia velocidades calculadas
            robot_controller.set_left_wheel_velocity(vl)
            robot_controller.set_right_wheel_velocity(vr)

            distance_goal = np.linalg.norm(np.array(goal_position[:2]) - np.array(robot_pos[:2]))

        print(f'Goal {goal_position} reached.')

    robot_controller.set_left_wheel_velocity(0)
    robot_controller.set_right_wheel_velocity(0)
    #robot_controller.stop_simulation()
    print('Program ended')
    
    plot_occupancy_grid(occupancy_grid, cols, rows, cell_size)


In [ ]:
def log_odds(p):
    return np.log(p / (1 - p))

def inv_log_odds(l):
    return 1 - (1 / (1 + np.exp(l)))

def inverse_range_sensor_model(cell, robot_cell, max_distance, cell_prob):
    # Modelo inverso de sensor
    # Para fins de demonstração, este modelo retorna uma diferença simples de distância
    return -0.4 if np.linalg.norm(cell - robot_cell) < max_distance else 0.4

def occupancy_grid_mapping2(cel_robo, cel_final, occupancy_grid):
    distanceMax = np.linalg.norm(cel_robo - cel_final)
    rr, cc = line(cel_robo[0], cel_robo[1], cel_final[0], cel_final[1])
    for i in range(len(rr)):
        cel_prob = occupancy_grid[rr[i], cc[i]]
        l_ant = log_odds(cel_prob)
        cel = np.array([rr[i], cc[i]])
        l_cell = l_ant + inverse_range_sensor_model(cel, cel_robo, distanceMax, cel_prob)
        occupancy_grid[rr[i], cc[i]] = inv_log_odds(l_cell)
    return occupancy_grid

# Parâmetros do grid de ocupação
map_size = [10, 10]  # Tamanho do mapa em metros
cell_size = 0.1  # Tamanho da célula em metros
rows, cols = int(map_size[0] / cell_size), int(map_size[1] / cell_size)  # Número de linhas e colunas

# Inicialização do occupancy grid
occupancy_grid = np.full((rows, cols), 0.5)  # Valor inicial 0.5 para representar células desconhecidas

# Conectando-se ao CoppeliaSim
clientID = sim.startSimulation()

if clientID != -1:
    print('Program Started')
    
    robot_controller = RobotController('kobuki')

    robot_controller.start_simulation()
    robot_pos = robot_controller.get_robot_position()
    robot_ori = robot_controller.get_robot_orientation()

    goal_position = [0.4, -4.45, 0]  # Posição inicial do objetivo
    robot_controller.set_goal_position(goal_position)
    
    # Handle para os dados do LASER
    laser_range_data = "hokuyo_range_data"
    laser_angle_data = "hokuyo_angle_data"
    
    distance_goal = np.linalg.norm(np.array(goal_position[:2]) - np.array(robot_pos[:2]))
    
    while distance_goal > 0.5:
        # Leitura do laser
        if laser_range_data is not None and laser_angle_data is not None:
            raw_range_data, raw_angle_data = readSensorData() 
            
        laser_data = np.array([raw_angle_data, raw_range_data]).T
    
        laser_transform = Rz(np.pi / 2)
        noisy_data = get_all_laser_data_ruido(laser_data, laser_transform)

        # Atualizando robo
        robot_pos = robot_controller.get_robot_position()
        robot_ori = robot_controller.get_robot_orientation()
        
        for i in range(len(laser_data)):
            fim_laser = Rz(np.pi/2) @ [(np.array(laser_data[i][0]) + map_size[0]/2), (np.array(laser_data[i][1]) + map_size[1]/2), 0]
            robo_mapa = Rz(np.pi/2) @ [(robot_pos[0] + map_size[0]/2), (robot_pos[1] + map_size[1]/2), 0]
            max_cel = len(occupancy_grid) - 1
            robot_x = min(max_cel, (robo_mapa[0]/cell_size).astype(int)) # posicao x do robo na celula do mapa occ grid
            robot_y = min(max_cel, (robo_mapa[1]/cell_size).astype(int)) # posicao y do robo na celula do mapa occ grid
            celula_x = min(max_cel, (fim_laser[0]/cell_size).astype(int)) # posicao x do laser na celula do mapa occ grid
            celula_y = min(max_cel, (fim_laser[1]/cell_size).astype(int))
            
            occupancy_grid = occupancy_grid_mapping2(np.array([robot_x, robot_y]), np.array([celula_x, celula_y]), occupancy_grid)
        
        Hlw = transformacao(robot_pos, robot_ori) # Matriz de transformação do laser para o mundo
        
        obs = [] # Lista de obstáculos
        obs_pts = [] # Lista de pontos dos obstáculos
        
        # Calculando forças
        Ft, Fatt, Frep = tt_force(np.array(robot_pos[:2]), np.array(goal_position[:2]), laser_data, obs, obs_pts, Hlw)
            
        dx, dy = Ft

        kr = 0.05
        kt = 0.1

        v = kr * (dx * np.cos(robot_ori[2]) + dy * np.sin(robot_ori[2]))
        w = kt * (np.arctan2(dy, dx) - robot_ori[2])

        v = max(min(v, 0.2), -0.2)
        w = max(min(w, np.deg2rad(45)), -np.deg2rad(45))

        vr = (2 * v + 0.23 * w) / (2 * 0.035)
        vl = (2 * v - 0.23 * w) / (2 * 0.035)

        # Envia velocidades calculadas
        robot_controller.set_left_wheel_velocity(vl)
        robot_controller.set_right_wheel_velocity(vr)

        distance_goal = np.linalg.norm(np.array(goal_position[:2]) - np.array(robot_pos[:2]))

    robot_controller.set_left_wheel_velocity(0)
    robot_controller.set_right_wheel_velocity(0)
    #robot_controller.stop_simulation()
    print('Program ended')
    
    plot_occupancy_grid(occupancy_grid, cols, rows, cell_size)
    plot_laser_points(noisy_data, robot_pos, robot_ori, map_size, cell_size)


In [ ]:


# Funções para o mapeamento de ocupação
def log_odds(prob):
    return np.log(prob / (1 - prob))

def inv_log_odds(lo):
    return 1 - 1 / (1 + np.exp(lo))

def update_cell(prob, log_odds_val):
    return inv_log_odds(log_odds(prob) + log_odds_val)

def new_inverse_range_sensor_model(cell, robot_cell, max_distance, l_0):
    x, y = cell
    x_robot, y_robot = robot_cell
    
    # Calcula a distância entre o robô e a célula
    r = np.sqrt((x - x_robot)**2 + (y - y_robot)**2)  # distância euclidiana
    
    max_dist = max_distance / cell_size
    alpha = 0.2 / cell_size
    distanceObs = np.linalg.norm(np.array(cell) - np.array(robot_cell[:2]))
    
    pocc = 0.85
    pfree = 0.1
    locc = log_odds(pocc)
    lfree = log_odds(pfree)

    if distanceObs > min(max_dist, max_distance + alpha / 2):  # Se a distância observada for maior que a distância máxima
        return log_odds(l_0)  # Retorna o log odds da célula anterior
    if max_distance < max_dist and abs(distanceObs - max_distance) <= alpha / 2:  # Se a distância observada for igual à distância máxima
        return locc  # Retorna o log odds da célula ocupada
    if distanceObs <= max_distance:  # Se a distância observada for menor que a distância máxima
        return lfree  # Retorna o log odds da célula livre

def bresenham(x0, y0, x1, y1):
    # Algoritmo de Bresenham para traçar linha
    cells = []
    dx = abs(x1 - x0)
    dy = abs(y1 - y0)
    sx = 1 if x0 < x1 else -1
    sy = 1 if y0 < y1 else -1
    err = dx - dy

    while True:
        cells.append((x0, y0))
        if x0 == x1 and y0 == y1:
            break
        e2 = 2 * err
        if e2 > -dy:
            err -= dy
            x0 += sx
        if e2 < dx:
            err += dx
            y0 += sy
    return cells

def new_occupancy_grid_mapping(occupancy_grid, robot_cell, laser_cells):
    """
    Atualiza o occupancy grid com base nas leituras do laser.
    
    Parameters:
    - occupancy_grid: Grid de ocupação a ser atualizado
    - robot_cell: Célula do robô (x, y)
    - laser_cells: Células do laser [(x, y), ...]
    """
    for point in laser_cells:
        laser_cell = world_to_grid(point, map_size, cell_size)
        cells = bresenham(robot_cell[0], robot_cell[1], laser_cell[0], laser_cell[1])
        for (t, i) in cells:
            if 0 <= t < occupancy_grid.shape[0] and 0 <= i < occupancy_grid.shape[1]:  # Verifica se está dentro dos limites do grid
                l_ant = occupancy_grid[t, i]
                l_cell = new_inverse_range_sensor_model([t, i], robot_cell, 5, 0.5)
                occupancy_grid[t, i] = update_cell(inv_log_odds(l_ant), l_cell)


# Cálculo do inverse sensor model
def inverse_range_sensor_model(cel, cel_robot, max_distance, prob):
    max = max_distance/cell_size
    alpha = 0.2/cell_size
    distanceObs = np.linalg.norm(cel - cel_robot[:2])
    pocc = 0.85
    pfree = 0.1
    locc = log_odds(pocc)
    lfree = log_odds(pfree)
    if distanceObs > min(max, max_distance + alpha/2):
        return log_odds(prob)
    if max_distance < max and abs(distanceObs - max_distance) <= alpha/2:
        return locc
    if distanceObs <= max_distance:
        return lfree

def new_occupancy_grid_mapping(occupancy_grid, robot_cell, laser_cells):
    distanceMax = np.linalg.norm(laser_cells[0] - robot_cell[:2])  # Corrigido
    for laser_cell in laser_cells:
        rr, cc = bresenham(robot_cell[0], robot_cell[1], laser_cell[0], laser_cell[1])
        for i in range(len(rr)):
            if 0 <= rr[i] < rows and 0 <= cc[i] < cols:  # Verificação de índices
                cel_prob = occupancy_grid[rr[i], cc[i]]
                l_ant = log_odds(cel_prob)
                l_cell = l_ant + inverse_range_sensor_model(cel, robot_cell, distanceMax, cel_prob)
                occupancy_grid[rr[i], cc[i]] = inv_log_odds(l_cell)
    return occupancy_grid

def occupancy_grid_mapping2(cel_robo, cel_final, occupancy_grid):
    distanceMax = np.linalg.norm(cel_robo - cel_final[0])  # Use apenas o primeiro laser_point
    
    for final_cell in cel_final:
        rr, cc = bresenham(cel_robo[0], cel_robo[1], final_cell[0], final_cell[1])
        for i in range(len(rr)):
            cel_prob = occupancy_grid[rr[i], cc[i]]
            l_ant = log_odds(cel_prob)
            cel = np.array([rr[i], cc[i]])
            l_cell = l_ant + inverse_range_sensor_model(cel, cel_robo, distanceMax, cel_prob)
            occupancy_grid[rr[i], cc[i]] = inv_log_odds(l_cell)
    
    return occupancy_grid